## 5.3.1 The Validation Set Approach

In [1]:
import numpy as np
import pandas as pd

pd.set_option('precision', 2) # number precision for pandas
pd.set_option('display.max_rows', 12)
pd.set_option('display.float_format', '{:20,.2f}'.format) # get rid of scientific notation

In [2]:
auto = pd.read_csv('datasets/Auto.csv', na_values=['?'])

In [3]:
auto

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.00,8,307.00,130.00,3504,12.00,70,1,chevrolet chevelle malibu
1,15.00,8,350.00,165.00,3693,11.50,70,1,buick skylark 320
2,18.00,8,318.00,150.00,3436,11.00,70,1,plymouth satellite
3,16.00,8,304.00,150.00,3433,12.00,70,1,amc rebel sst
4,17.00,8,302.00,140.00,3449,10.50,70,1,ford torino
5,15.00,8,429.00,198.00,4341,10.00,70,1,ford galaxie 500
...,...,...,...,...,...,...,...,...,...
391,27.00,4,151.00,90.00,2950,17.30,82,1,chevrolet camaro
392,27.00,4,140.00,86.00,2790,15.60,82,1,ford mustang gl
393,44.00,4,97.00,52.00,2130,24.60,82,2,vw pickup


In [6]:
auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 9 columns):
mpg             397 non-null float64
cylinders       397 non-null int64
displacement    397 non-null float64
horsepower      392 non-null float64
weight          397 non-null int64
acceleration    397 non-null float64
year            397 non-null int64
origin          397 non-null int64
name            397 non-null object
dtypes: float64(4), int64(4), object(1)
memory usage: 28.0+ KB


In [7]:
auto.dropna(axis=0, inplace=True)

In [8]:
auto.cylinders = auto.cylinders.astype('category')
auto.name = auto.name.astype('category')

auto.reset_index(inplace=True)

auto['horsepower_2'] = np.power(auto.horsepower, 2) #제곱
auto['horsepower_3'] = np.power(auto.horsepower, 3)
auto['horsepower_4'] = np.power(auto.horsepower, 4)
auto['horsepower_5'] = np.power(auto.horsepower, 5)

In [12]:
# Polynomial Features using sklearn:
from sklearn.preprocessing import PolynomialFeatures
pol = PolynomialFeatures(degree=5, interaction_only=False, include_bias=False)
polf = pol.fit_transform(auto.loc[:, 'horsepower'].values.reshape(-1, 1))

In [14]:
from sklearn.model_selection import train_test_split

X, y = auto.loc[:, ['horsepower', 'horsepower_2', 'horsepower_3']], auto.mpg

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# ols model with intercept
lm1 = LinearRegression(fit_intercept=True)
lm2 = LinearRegression(fit_intercept=True)
lm3 = LinearRegression(fit_intercept=True)

lm1_fit = lm1.fit(X_train.loc[:, 'horsepower'].values.reshape(-1, 1), y_train)
lm2_fit = lm2.fit(X_train.loc[:, ['horsepower', 'horsepower_2']], y_train)
lm3_fit = lm3.fit(X_train.loc[:, ['horsepower', 'horsepower_2', 'horsepower_3']], y_train)

lm1_predict = lm1_fit.predict(X_test.loc[:, 'horsepower'].values.reshape(-1, 1))
lm2_predict = lm2_fit.predict(X_test.loc[:, ['horsepower', 'horsepower_2']])
lm3_predict = lm3_fit.predict(X_test.loc[:, ['horsepower', 'horsepower_2', 'horsepower_3']])

print('lm1 MSE:', mean_squared_error(y_test, lm1_predict))
print('lm2 MSE:', mean_squared_error(y_test, lm2_predict))
print('lm3 MSE:', mean_squared_error(y_test, lm3_predict))

lm1 MSE: 25.5738781896844
lm2 MSE: 22.218020050032862
lm3 MSE: 22.66767543553449


These results are consistent with our previous ﬁndings: a model that predicts mpg using a quadratic function of horsepower performs better than a model that involves only a linear function of horsepower, and there is little evidence in favor of a model that uses a cubic function of horsepower.

## 5.3.2 Leave-One-Out Cross-Validation

$$ CV_{\left( n \right)} = \frac{1}{n} \sum_{i=1}^n MSE_i $$

In [16]:
from sklearn.model_selection import LeaveOneOut

X, y = auto.loc[:, ['horsepower', 'horsepower_2', 'horsepower_3', 'horsepower_4', 'horsepower_5']], auto.mpg

loocv = LeaveOneOut()
loocv.get_n_splits(X)

392

In [18]:
loocv_mse = []
lm = LinearRegression(fit_intercept=True)

for train_index, test_index in loocv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    lm1_fit = lm.fit(X_train.loc[:, 'horsepower'].values.reshape(-1, 1), y_train)
    lm1_predict = lm1_fit.predict(X_test.loc[:, 'horsepower'].values.reshape(-1, 1))
    
    loocv_mse.append(mean_squared_error(y_test, lm1_predict))
    
np.array(loocv_mse).mean()

24.231513517929226

In [19]:
# using sklearn cross_validation_score
from sklearn.model_selection import cross_val_score

lm = LinearRegression(fit_intercept=True)

cval = cross_val_score(lm, 
                       auto.loc[:, 'horsepower'].values.reshape(-1, 1), 
                       auto.mpg, 
                       cv=len(auto), # k=n인 k-Fold가 바로 LOOCV
                       n_jobs=-1, 
                       scoring='neg_mean_squared_error')

In [20]:
cval.mean() #-cval.mean()

-24.23151351792923

In [21]:
# Loop for 5 degree polinomial linear regressions with LOOCV
loocv_poly = {}

for i in range(1, 6):
    
    loocv_mse = []
    
    for train_index, test_index in loocv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        if i == 1:
            X_TRAIN = X_train.iloc[:,0:1].values.reshape(-1, 1)
            X_TEST = X_test.iloc[:,0:1].values.reshape(-1, 1)
        else:
            X_TRAIN = X_train.iloc[:,0:i]
            X_TEST = X_test.iloc[:,0:i]
            
        loocv_mse.append(mean_squared_error(y_test, LinearRegression(fit_intercept=True).fit(X_TRAIN, y_train).predict(X_TEST)))
        
        loocv_poly['lm' + str(i) + '_MSE'] = np.array(loocv_mse).mean()

In [22]:
loocv_poly

{'lm1_MSE': 24.231513517929226,
 'lm2_MSE': 19.248213124489677,
 'lm3_MSE': 19.334984064029538,
 'lm4_MSE': 19.424430310310466,
 'lm5_MSE': 19.033212804000605}

## 5.3.3 k-Fold Cross-Validation

loocv는 k-fold의 특이 케이스(k=n)

In [23]:
from sklearn.model_selection import KFold

X, y = auto.loc[:, ['horsepower', 'horsepower_2', 'horsepower_3', 'horsepower_4', 'horsepower_5']], auto.mpg

kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [24]:
# Loop for 5 degree polinomial linear regressions with k-Fold CV

kf_poly = {}

for i in range(1, 6):
    
    kf_mse = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        if i == 1:
            X_TRAIN = X_train.iloc[:,0:1].values.reshape(-1, 1)
            X_TEST = X_test.iloc[:,0:1].values.reshape(-1, 1)
        else:
            X_TRAIN = X_train.iloc[:,0:i]
            X_TEST = X_test.iloc[:,0:i]
        
        kf_mse.append(
            mean_squared_error(
                y_test, 
                LinearRegression(fit_intercept=True)
                .fit(
                    X_TRAIN, 
                    y_train
                )
                .predict(
                    X_TEST
                )))
        
        kf_poly['lm' + str(i) + '_MSE'] = np.array(kf_mse).mean()

In [25]:
kf_poly

{'lm1_MSE': 24.199808197692477,
 'lm2_MSE': 19.22863661426802,
 'lm3_MSE': 19.26626534663196,
 'lm4_MSE': 19.35109227305059,
 'lm5_MSE': 19.02323300932039}

## 5.3.4 The Bootstrap

In [26]:
portfolio = pd.read_csv('datasets/Portfolio.csv', index_col=0)

In [27]:
portfolio

,X,Y
1,-0.90,-0.23
2,-1.56,-0.89
3,-0.42,0.27
4,1.04,-0.73
5,-0.32,0.84
6,-1.74,-2.04
...,...,...
95,2.26,0.67
96,0.48,1.45
97,-0.54,-0.40


In [28]:
def alpha_fn(data, start_index, end_index):
    X = data['X'][start_index:end_index]
    Y = data['Y'][start_index:end_index]
    return ((np.var(Y) - np.cov(X, Y)[0][1]) / (np.var(X) + np.var(Y) - 2 * np.cov(X, Y)[0][1]))

$$ \alpha = \frac{\sigma_Y^2 - \sigma_{XY}}{\sigma_X^2 + \sigma_Y^2 - 2 \sigma_{XY}} $$

In [29]:
alpha_fn(portfolio, 0, 100) # use all 100 obs일때

0.5766511516104118

In [30]:
# use sample function for bootstrap sampling 
#중복 허용 랜덤
from sklearn.utils import resample

portfolio_bs = resample(portfolio, replace=True, n_samples=100)
alpha_fn(portfolio_bs, 0, 100)

0.48955576946792495

In [31]:
bs_alpha = []

for i in range(0, 1000):
    bs_alpha.append(alpha_fn(resample(portfolio, replace=True, n_samples=100), 0, 100))

bs_alpha = np.array(bs_alpha)

print('Bootstrapped alpha:', bs_alpha.mean())
print('SE:',  bs_alpha.std())

Bootstrapped alpha: 0.5808457000125028
SE: 0.09267636973659929


In [32]:
# estimates of parameters usual linear regression
def boot_fn(data, start_index, end_index):
    m = LinearRegression(fit_intercept=True).fit(
        data['horsepower'][start_index:end_index].values.reshape(-1, 1),
        data['mpg'][start_index:end_index])
    
    return m.intercept_, m.coef_
    
boot_fn(auto, 0, 392)

(39.93586102117047, array([-0.15784473]))

In [33]:
boot_fn(resample(auto, replace=True, n_samples=392), 0, 392)

(40.58035039762748, array([-0.16312637]))

In [35]:
# create bootstrap estimates 
boot_fn(resample(auto, replace=True, n_samples=392), 0, 392)

(39.83062220275909, array([-0.15733682]))

In [36]:
# compute SE of 1000 bootstrap estimates
bs_boot = {'t1': [], 't2': []}

for i in range(0, 1000):
    bs_boot['t1'].append(boot_fn(resample(auto, replace=True, n_samples=392), 0, 392)[0])
    bs_boot['t2'].append(boot_fn(resample(auto, replace=True, n_samples=392), 0, 392)[1][0])

t1_es = np.array(bs_boot['t1']).mean()
t1_se = np.array(bs_boot['t1']).std()
t2_es = np.array(bs_boot['t2']).mean()
t2_se = np.array(bs_boot['t2']).std()

print('t1 bs estimate & se:', t1_es, t1_se)
print('t2 bs estimate & se:', t2_es, t2_se)

t1 bs estimate & se: 39.976437608451405 0.8604305306018767
t2 bs estimate & se: -0.15787683364545693 0.0076319574950846365


In [37]:
import statsmodels.formula.api as sm

ols = sm.ols('mpg ~ horsepower', data=auto).fit()
ols.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,39.9359,0.717,55.660,0.000,38.525,41.347
horsepower,-0.1578,0.006,-24.489,0.000,-0.171,-0.145


In [38]:
def boot_fn2(data, start_index, end_index):
    m = LinearRegression(fit_intercept=True).fit(
        data[['horsepower', 'horsepower_2']][start_index:end_index], #horsepower2 추가
        data['mpg'][start_index:end_index])
    
    return m.intercept_, m.coef_

In [39]:
bs_boot2 = {'t1': [], 't2': [], 't3': []}

for i in range(0, 1000):
    bs_boot2['t1'].append(
        boot_fn2(resample(auto, replace=True, n_samples=392), 0, 392)[0]
    )
    bs_boot2['t2'].append(
        boot_fn2(resample(auto, replace=True, n_samples=392), 0, 392)[1][0]
    )
    bs_boot2['t3'].append(
        boot_fn2(resample(auto, replace=True, n_samples=392), 0, 392)[1][1]
    )


t1_es = np.array(bs_boot2['t1']).mean()
t1_se = np.array(bs_boot2['t1']).std()
t2_es = np.array(bs_boot2['t2']).mean()
t2_se = np.array(bs_boot2['t2']).std()
t3_es = np.array(bs_boot2['t3']).mean()
t3_se = np.array(bs_boot2['t3']).std()

print('t1 bs estimate & se:', t1_es, t1_se)
print('t2 bs estimate & se:', t2_es, t2_se)
print('t3 bs estimate & se:', t3_es, t3_se)

t1 bs estimate & se: 56.91865138430965 2.1543994998746396
t2 bs estimate & se: -0.4669285837759115 0.03418949505171125
t3 bs estimate & se: 0.0012299970171830358 0.0001196071570809738


In [40]:
import statsmodels.formula.api as sm

ols2 = sm.ols('mpg ~ horsepower + horsepower_2', data=auto).fit()
ols2.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,56.9001,1.800,31.604,0.000,53.360,60.440
horsepower,-0.4662,0.031,-14.978,0.000,-0.527,-0.405
horsepower_2,0.0012,0.000,10.080,0.000,0.001,0.001
